In [ ]:
pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from lightfm.data import Dataset

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#author -  user_id
#appid - game
#voted_up - true means it was a positive recommendation
#votes_up - the number of users that found this review helpful
# votes_up(integer)

# data_overten = pd.read_csv('/content/drive/My Drive/steam/parsed_data_overten1.csv')
data_overten = {'author' : ['1','2', '3', '4', '5', '6', '7'],
                'appid' : ['10', '10', '20', '20', '30', '30', '30'],
                'playtime_forever' : ['104', '426', '386', '165', '28', '45', '30'], 
                'playtime_at_review' : ['10', '426', '199', '184', '40', '21', '31'],
                'voted_up' : ['True', 'Ture', 'Ture', 'Ture', 'False', 'False', 'False'],
                'votes_up' : [0, 0, 3, 4, 1, 2, 1]}
data_user_game = pd.DataFrame(data_overten, columns = ['author', 'appid', 'votes_up'])
features = pd.DataFrame(data_overten, columns = ['author', 'playtime_forever', 'playtime_at_review'])

# Calling the fit method


In [ ]:
uf = [] 
col = ['playtime_forever']*len(features.playtime_forever.unique()) + ['playtime_at_review']*len(features.playtime_at_review.unique()) 
unique_f1 = list(features.playtime_forever.unique()) + list(features.playtime_at_review.unique())  
#print('f1:', unique_f1) 
for x,y in zip(col, unique_f1): 
    res = x + ":" + y
    uf.append(res) 
    print(res)

playtime_forever:104
playtime_forever:426
playtime_forever:386
playtime_forever:165
playtime_forever:28
playtime_forever:45
playtime_forever:30
playtime_at_review:10
playtime_at_review:426
playtime_at_review:199
playtime_at_review:184
playtime_at_review:40
playtime_at_review:21
playtime_at_review:31


# Creating LightFM-compliant dataset & fit method

In [ ]:
dataset1 = Dataset()
dataset1.fit(data_user_game['author'].unique(),
             data_user_game['appid'].unique(),
             user_features = uf)

# Building Interactions
iterable of interactions where each interaction is a tuple containing three elements : 
1.   user
2.   item
3.   interaction weights

In [ ]:
# interactions : user가 item과 interaction했는지 여부
# weights : 해당 특정 interaction정량화
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in data_user_game.values])

In [ ]:
interactions.todense()

matrix([[1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]], dtype=int32)

In [ ]:
weights.todense()

matrix([[0., 0., 0.],
        [0., 0., 0.],
        [0., 3., 0.],
        [0., 4., 0.],
        [0., 0., 1.],
        [0., 0., 2.],
        [0., 0., 1.]], dtype=float32)

# Building user features

In [ ]:
# inputs to the build_user_features should be like 
# [ 
#      ('1', ['playtime_forever:104', 'playtime_at_review:10']), 
#      ......
#      ('7', ['playtime_forever:30', 'playtime_at_review:31'])
# ]
# user features를 가져와 feature:value 형식으로 변환하도록 도와주는 함수 
def feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [104,10],
    resultant output = ['playtime_forever:104', 'playtime_at_review:10']
   
    """
    result = []
    ll = ['playtime_forever:','playtime_at_review:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = x +""+ y
        result.append(res)
    return result

In [ ]:
# feature_colon_value함수를 이용 모든 이용자의 feature을 바른 형식으로 생성
ad_subset = features[['playtime_forever','playtime_at_review']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(feature_colon_value(item))
print(f'Final output: {feature_list}')

Final output: [['playtime_forever:104', 'playtime_at_review:10'], ['playtime_forever:426', 'playtime_at_review:426'], ['playtime_forever:386', 'playtime_at_review:199'], ['playtime_forever:165', 'playtime_at_review:184'], ['playtime_forever:28', 'playtime_at_review:40'], ['playtime_forever:45', 'playtime_at_review:21'], ['playtime_forever:30', 'playtime_at_review:31']]


In [ ]:
# 각 요소를 feature_list 해당 사용자의 id와 연결
user_tuple = list(zip(features.author, feature_list))
user_tuple

[('1', ['playtime_forever:104', 'playtime_at_review:10']),
 ('2', ['playtime_forever:426', 'playtime_at_review:426']),
 ('3', ['playtime_forever:386', 'playtime_at_review:199']),
 ('4', ['playtime_forever:165', 'playtime_at_review:184']),
 ('5', ['playtime_forever:28', 'playtime_at_review:40']),
 ('6', ['playtime_forever:45', 'playtime_at_review:21']),
 ('7', ['playtime_forever:30', 'playtime_at_review:31'])]

In [ ]:
# user_features행렬에서 행은 사용자
# 열은 사용자의 feature, user가 feature을 가질 때마다 1이 존재 
user_features = dataset1.build_user_features(user_tuple, normalize= False)
user_features.todense()

matrix([[1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
# 열이 21개인 이유 사용자의 id도 feature이므로 
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset1.mapping()
dataset1.mapping()

({'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6},
 {'1': 0,
  '2': 1,
  '3': 2,
  '4': 3,
  '5': 4,
  '6': 5,
  '7': 6,
  'playtime_forever:104': 7,
  'playtime_forever:426': 8,
  'playtime_forever:386': 9,
  'playtime_forever:165': 10,
  'playtime_forever:28': 11,
  'playtime_forever:45': 12,
  'playtime_forever:30': 13,
  'playtime_at_review:10': 14,
  'playtime_at_review:426': 15,
  'playtime_at_review:199': 16,
  'playtime_at_review:184': 17,
  'playtime_at_review:40': 18,
  'playtime_at_review:21': 19,
  'playtime_at_review:31': 20},
 {'10': 0, '20': 1, '30': 2},
 {'10': 0, '20': 1, '30': 2})

In [ ]:
user_feature_map

{'1': 0,
 '2': 1,
 '3': 2,
 '4': 3,
 '5': 4,
 '6': 5,
 '7': 6,
 'playtime_forever:104': 7,
 'playtime_forever:426': 8,
 'playtime_forever:386': 9,
 'playtime_forever:165': 10,
 'playtime_forever:28': 11,
 'playtime_forever:45': 12,
 'playtime_forever:30': 13,
 'playtime_at_review:10': 14,
 'playtime_at_review:426': 15,
 'playtime_at_review:199': 16,
 'playtime_at_review:184': 17,
 'playtime_at_review:40': 18,
 'playtime_at_review:21': 19,
 'playtime_at_review:31': 20}

# Training the model

In [ ]:
from scipy import sparse
from lightfm import LightFM
# wrap : 랭킹 손실 함수, 실제로 구매한 상품에 대한 구매 확률보다 실제로 구매하지 않은 상품에 대한 
# 구매 확룰이 더 큰 경우(error)에만 모델 업데이트를 수행해가며 최종 실제로 구매한 상품의 구매 확률을 가장 크게 만드는 것
model = LightFM(loss='warp')
model.fit(interactions, # spase matrix representing whether user u and item i interacted
      user_features= user_features, # we have built the sparse matrix above
      sample_weight= weights, # spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
      epochs=10)

# Evaluating the model
AUC는 ROC(Receiver Operating Charateristic)곡선 아래의 면적을 뜻하며, binary classification or multi-class classification문제에서 True라벨과 flase라벨의 분류 성능을 측정하는 지표이다.
아래 그래프 이미지는 FPR(False Positive Rate)과 TPR(True Positive Rate)을 각각 x축, y축으로 하는 ROC곡선을 그린 그래프이다. ROC곡선 아래 ('A')이 AUC score이며, AUC score가 1에 가까울 수록 좋은 분류 성능을 가진 모델이다.

<img src = "https://drive.google.com/uc?id=1l8Be823kn8ZcAcWjqmGK8GqJRWxJg3Zy" height = 230 width = 300>


In [ ]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model,
                      interactions,
                      user_features=user_features
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

Hybrid training set AUC: 0.5


# Prediction for KNOWN user

In [ ]:
# predict for existing user
# ex) user 1 = '0', user 2 = '1', user 3 = '2'
user_x = user_id_map['2']
n_users, n_items = interactions.shape # no of users * no of items
model.predict(user_x, np.arange(n_items)) # means predict for all users

array([-0.41044816,  0.35298854,  0.04352101], dtype=float32)

# Prediction for NEW user

In [ ]:
# predict for new user
user_feature_list = ['playtime_forever:100', 'playtime_at_review:100']

In [ ]:
def format_newuser_input(user_feature_map, user_feature_list):
  num_features = len(user_feature_list)
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass

  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [ ]:
new_user_features = format_newuser_input(user_feature_map, user_feature_list)
new_user_features.todense()

new user feature encountered 'playtime_forever:100'
new user feature encountered 'playtime_at_review:100'


matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]])

In [ ]:
model.predict(0, np.arange(n_items), user_features=new_user_features) # Here 0 means pick the first row of t

array([-0.41100636,  0.3518061 ,  0.04333792], dtype=float32)